Написать приложение,
которое собирает основные новости с сайтов:
    
    mail.ru, lenta.ru, yandex-новости.
Для парсинга использовать XPath.
Структура данных должна содержать:

    название источника;
    наименование новости;
    ссылку на новость;
    дата публикации.

Сложить все в базу данных

In [1]:
from pprint import pprint
from lxml import html
import requests
import datetime
from pymongo import MongoClient, errors

In [2]:

def get_news(website: str, rubric: str):
    """
    website == 'mail.ru' ,
               'lenta.ru',
               'yandex.ru'
              
    rubric == 'politics',
              'economics',
              'society',
              'incident'
    """
    rubric_dict = {'politics': 0, 'economics': 1, 'society': 2, 'incident': 3}
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'}
    links_for_news = {'mail.ru' : {'main_link': 'https://news.mail.ru',
                                   'second_link': ['/politics/', '/economics/', '/society/', '/incident/']},
                      'lenta.ru': {'main_link': 'https://lenta.ru',
                                   'second_link': ['/rubrics/world/politic/', '/rubrics/economics/', '/rubrics/world/society/', '/rubrics/world/accident/']},
                      'yandex.ru': {'main_link': 'https://yandex.ru/news',
                                    'second_link': ['/rubric/politics', '/rubric/business', '/rubric/society', '/rubric/incident']}
                     }

    news_link_xpath = {'mail.ru' : "//div[@data-module='TrackBlocks']//a[contains(@href, 'news.mail.ru') and not(contains(@href, 'comments'))]/@href",
                       'lenta.ru': "//section[contains(@class, 'rubric')]//a[@href and not(contains(@class, 'banner-pay'))]/@href",
                       'yandex.ru': "//article[contains(@class, 'news-card')]//a[@class='news-card__link']/@href"}
    news_name_xpath = {'mail.ru' : "//div[@data-module='TrackBlocks']/div[contains(@class,'hdr') and contains(@class,'title')]//h1",
                       'lenta.ru': "//div[contains(@class,'b-topic-layout')]//h1",
                       'yandex.ru': "//article[contains(@class, 'news-card')]//a[@class='news-card__link']/h2"}
    date_of_publication_xpath = {'mail.ru' : "//div[@data-module='TrackBlocks']/div[contains(@class,'breadcrumbs')]//span[@datetime]/@datetime",
                                 'lenta.ru': "//div[contains(@class,'b-topic-layout')]//time[@class='g-date']/@datetime",
                                 'yandex.ru': "//article[contains(@class, 'news-card')]//span[@class='mg-card-source__time']"}                           
    news_source_link_xpath = {'mail.ru' : "//div[@data-module='TrackBlocks']/div[contains(@class,'breadcrumbs')]//a[@target]/@href",
                              'lenta.ru': "https://lenta.ru",
                              'yandex.ru': "//article[contains(@class, 'news-card')]//div[contains(@class, 'news-card__source')]/span/a[@href]/@href"}
    news_source_name_xpath = {'mail.ru' : "//div[@data-module='TrackBlocks']/div[contains(@class,'breadcrumbs')]//a[@target]/span[@class='link__text']",
                              'lenta.ru': "Lenta.ru («Лента.ру»)",
                              'yandex.ru': "//article[contains(@class, 'news-card')]//div[contains(@class, 'news-card__source')]//a/text()"}
    
    link = links_for_news[website]['main_link'] + links_for_news[website]['second_link'][rubric_dict[rubric]]
    request = requests.get(link, headers=headers)
    root_dom = html.fromstring(request.text)
    news_links = root_dom.xpath(news_link_xpath[website])
    news_data = []
    if website == 'yandex.ru':
        news_name = root_dom.xpath(news_name_xpath[website])
        date_of_publication = root_dom.xpath(date_of_publication_xpath[website])
        news_source_link = root_dom.xpath(news_source_link_xpath[website])
        news_source_name = root_dom.xpath(news_source_name_xpath[website])
        for link_item in  range(len(news_links)):
            news = {}
            news['news_link'] = news_links[link_item]
            news['news_name'] = news_name[link_item].text
            __date_news = datetime.date.today()
            news['date_of_publication'] =   [f'{str(__date_news)} {date_of_publication[link_item].text}']
            request = requests.get(news_source_link[link_item], headers=headers)
            root_dom = html.fromstring(request.text)
            news['news_source_link'] = root_dom.xpath("//a[@class='news-story__subtitle']/@href")[0]
            news['news_source_name'] = news_source_name[link_item]
            news_data.append(news)
        return news_data
    
    for link_item in set(news_links):
        news = {}
        if website == 'lenta.ru':
            link_item = links_for_news[website]['main_link'] + link_item
        request = requests.get(link_item, headers=headers)
        root_dom = html.fromstring(request.text)
        
        news_name = root_dom.xpath(news_name_xpath[website])
        date_of_publication = root_dom.xpath(date_of_publication_xpath[website])
        
        news['news_link'] = link_item
        news['date_of_publication'] = date_of_publication
        if website == 'mail.ru':
            news['news_name'] = news_name[0].text
            
            news_source_link = root_dom.xpath(news_source_link_xpath[website])
            news_source_name = root_dom.xpath(news_source_name_xpath[website])
            news['news_source_link'] = news_source_link[0]
            news['news_source_name'] = news_source_name[0].text
        elif website == 'lenta.ru':        
            news['news_name'] = news_name[0].text.replace('\xa0', ' ')  
            news['news_source_link'] = news_source_link_xpath[website]
            news['news_source_name'] = news_source_name_xpath[website]
        
        news_data.append(news)
        
    return news_data
        

In [3]:
mail_politics_news = get_news('mail.ru', rubric='politics')
mail_politics_news

[{'news_link': 'https://news.mail.ru/politics/43337955/',
  'date_of_publication': ['2020-09-12T22:15:50+03:00'],
  'news_name': 'Как прошел второй день досрочного голосования в Ленобласти',
  'news_source_link': 'https://ivbg.ru/',
  'news_source_name': 'ivbg.ru'},
 {'news_link': 'https://news.mail.ru/politics/43345991/',
  'date_of_publication': ['2020-09-13T18:12:19+03:00'],
  'news_name': 'Основная масса протестующих в Минске собралась в районе Дрозды',
  'news_source_link': 'http://www.tass.ru/',
  'news_source_name': 'ТАСС'},
 {'news_link': 'https://news.mail.ru/politics/43345119/',
  'date_of_publication': ['2020-09-13T16:18:46+03:00'],
  'news_name': 'Канцлер Австрии сообщил о начале второй волны коронавируса в стране',
  'news_source_link': 'http://www.kommersant.ru',
  'news_source_name': 'Коммерсантъ'},
 {'news_link': 'https://news.mail.ru/politics/43328260/',
  'date_of_publication': ['2020-09-11T17:27:40+03:00'],
  'news_name': 'Правительство переименовало Минкомсвязи',
  

In [4]:
lenta_politics_news = get_news('lenta.ru', rubric='politics')
lenta_politics_news

[{'news_link': 'https://lenta.ru/news/2020/09/12/mcgregor/',
  'date_of_publication': ['2020-09-12T19:43:00+03:00'],
  'news_name': 'Макгрегора арестовали',
  'news_source_link': 'https://lenta.ru',
  'news_source_name': 'Lenta.ru («Лента.ру»)'},
 {'news_link': 'https://lenta.ru/news/2020/09/13/mavzolee/',
  'date_of_publication': ['2020-09-13T18:45:00+03:00'],
  'news_name': 'Объяснено возникновение идеи поместить тело Ленина в Мавзолей',
  'news_source_link': 'https://lenta.ru',
  'news_source_name': 'Lenta.ru («Лента.ру»)'},
 {'news_link': 'https://lenta.ru/news/2020/09/12/domoy/',
  'date_of_publication': ['2020-09-12T21:00:00+03:00'],
  'news_name': 'Палестина отозвала посла в Бахрейне из-за примирения с Израилем',
  'news_source_link': 'https://lenta.ru',
  'news_source_name': 'Lenta.ru («Лента.ру»)'},
 {'news_link': 'https://lenta.ru/news/2020/09/13/ugroza/',
  'date_of_publication': ['2020-09-13T14:01:00+03:00'],
  'news_name': 'Китай признал США главной угрозой международному 

In [5]:
yandex_politics_news = get_news('yandex.ru', rubric='politics')
yandex_politics_news

[{'news_link': 'https://yandex.ru/news/story/V_Britanii_zayavili_ob_opasnosti_rossijskoj_rakety_Burevestnik--fdaeeef623029fd04a75b4021191354b?lang=ru&rubric=politics&stid=hlprKi5qIanC_5yKNsxL&t=1600020238&tt=true&persistent_id=112335242',
  'news_name': 'В Британии заявили об опасности российской ракеты «Буревестник»',
  'date_of_publication': ['2020-09-13 20:38'],
  'news_source_link': 'https://www.rbc.ru/politics/13/09/2020/5f5e35e89a7947673e94b035?utm_source=yxnews&utm_medium=desktop',
  'news_source_name': 'РБК'},
 {'news_link': 'https://yandex.ru/news/story/Ukraina_obyasnila_otkaz_ot_postavok_vody_v_Krym--8c68f662bae0b1c534a0699a898cba4e?lang=ru&rubric=politics&stid=DkBUdq0Sax17Nhfbg-n3&t=1600020238&tt=true&persistent_id=112336874',
  'news_name': 'Украина объяснила отказ от поставок воды в Крым',
  'date_of_publication': ['2020-09-13 21:01'],
  'news_source_link': 'https://www.rbc.ru/rbcfreenews/5f5e32799a7947667fd03ce2?utm_source=yxnews&utm_medium=desktop',
  'news_source_name':

In [6]:
data_insert = mail_politics_news
data_insert.extend(lenta_politics_news)
data_insert.extend(yandex_politics_news)
data_insert

[{'news_link': 'https://news.mail.ru/politics/43337955/',
  'date_of_publication': ['2020-09-12T22:15:50+03:00'],
  'news_name': 'Как прошел второй день досрочного голосования в Ленобласти',
  'news_source_link': 'https://ivbg.ru/',
  'news_source_name': 'ivbg.ru'},
 {'news_link': 'https://news.mail.ru/politics/43345991/',
  'date_of_publication': ['2020-09-13T18:12:19+03:00'],
  'news_name': 'Основная масса протестующих в Минске собралась в районе Дрозды',
  'news_source_link': 'http://www.tass.ru/',
  'news_source_name': 'ТАСС'},
 {'news_link': 'https://news.mail.ru/politics/43345119/',
  'date_of_publication': ['2020-09-13T16:18:46+03:00'],
  'news_name': 'Канцлер Австрии сообщил о начале второй волны коронавируса в стране',
  'news_source_link': 'http://www.kommersant.ru',
  'news_source_name': 'Коммерсантъ'},
 {'news_link': 'https://news.mail.ru/politics/43328260/',
  'date_of_publication': ['2020-09-11T17:27:40+03:00'],
  'news_name': 'Правительство переименовало Минкомсвязи',
  

In [7]:
len(data_insert)

71

In [8]:
client = MongoClient('127.0.0.1:27017',
                     username='admin_news',
                     password='password_db',
                     authSource='news_db',
                     authMechanism='SCRAM-SHA-1')


db = client['news_db']
collection = db.news

count = 0
for item_news in data_insert:
    try:
        collection.insert_one(item_news)
        count += 1
    except errors.DuplicateKeyError:
        print("Errdor DuplicateKeyError")

count

71